# Γιάννης Δαλιάνης
# 1115201700027
# Homework 2
# Άσκηση 5

In [ ]:
import os
import re
from google.colab import drive
import pandas as pd

import nltk
nltk.download('stopwords')

from sklearn.preprocessing import LabelEncoder, MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.ERROR)

import random
import numpy as np

import spacy
# nlp = spacy.load('en')
nlp = spacy.load("en_core_web_sm")

from matplotlib import pyplot as plt

import torch.nn.functional as F

import time

!pip install transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertModel
from transformers import BertTokenizer

import torch   
from torchtext import data, datasets
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

Drop uneccessary columns.

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

tweets = pd.read_csv('/content/gdrive/My Drive/DI/runAI2/ex2/SentimentTweets.csv', index_col=0)

print(tweets.columns)

tweets = tweets.drop(['id', 'date', 'flag', 'user'], axis=1)
tweets.rename(columns={'target': 'label'}, inplace = True)

tweets

In [ ]:
# tweets = tweets.sample(n = 1000)

Clean tweet text, remove stopwords and save to new csv file.

In [ ]:
tweets = tweets.apply(lambda x: x.astype(str).str.lower())

def cleanText(text):
    text = text.str.replace(r'RT[\s]+', '')                                              # Removing RT
    text = text.str.replace(r'&amp;', '&')            # Replace '&amp;' with '&'
    text = text.str.replace(r'#.*?(?=\s|$)', '')                                        # remove hashtags and mentions
    text = text.str.replace(r'@.*?(?=\s|$)', '')
    text = text.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)  # remove urls
    text = text.apply(lambda x: re.split('https:\/\/.*', str(x))[0])
    text = text.str.replace(r"\'re", " are")                                             # Change 're to 'are'
    text = text.str.replace(r"\'t", " not")                                             # Change 't to 'not'
    text = text.str.replace(r"\'d", " would")                                           # Change 'd to 'would'
    text = text.replace(r'\\n',' ', regex=True)                                         # remove newlines and other special characters
    text = text.replace(r'\\u',' ', regex=True)
    text = text.replace(r'\\x',' ', regex=True)
    text = text.str.replace('\d+', '')                                                  # remove all numbers
    text = text.str.replace(r'\b(\w{1,2})\b', '')                                       # remove words with 2 or 1 letter only
    text = text.str.replace('[^\w\s]','')                                               # remove punctuations
    text = text.apply(lambda x: re.sub(' +', ' ', x))                                   # replace multiple whitespaces
    # text = text.str.replace(r'(@.*?)[\s]', '')        # Remove '@name'
    # text = text.str.replace(r'\s+', ' ').str.strip()  # Remove trailing whitespace
    return text

def remove_stops(row):
  stops = nltk.corpus.stopwords.words("english")
  meaningful_words = [w for w in row if w not in stops]
  return meaningful_words

def rejoin_words(row):
  joined_words = ( " ".join(row))
  return joined_words

tweets['text'] = cleanText(tweets['text'])

# erase empty lines
nan_value = float("NaN")
tweets['text'].replace(" ", "", inplace=True)
tweets['text'].replace("", nan_value, inplace=True)
tweets.dropna(subset=['text'], inplace=True)

# 0 will be for negative and 1 for positive
tweets['label'] = tweets['label'].astype(int)
tweets['label'].replace(4, 1, inplace=True)

# takes some time
tweets["text"] = tweets["text"].str.split()
tweets['text'] = tweets['text'].apply(lambda x: remove_stops(x))
tweets['text'] = tweets['text'].apply(lambda x: rejoin_words(x))

# erase empty lines
nan_value = float("NaN")
tweets['text'].replace(" ", "", inplace=True)
tweets['text'].replace("", nan_value, inplace=True)
tweets.dropna(subset=['text'], inplace=True)

tweets.reset_index(drop = True, inplace = True)

print(tweets.columns)

trainTWEETS, testTWEETS = train_test_split(tweets, test_size=0.2)

# trainTWEETS.to_csv(r'/content/gdrive/My Drive/DI/trainTWEETS.csv')
# testTWEETS.to_csv(r'/content/gdrive/My Drive/DI/testTWEETS.csv')

tweets

# SECOND PART


In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
start_timeTOTAL = time.time()

Takes time but has awesome results

In [ ]:
# trainDATA = pd.read_csv('/content/gdrive/My Drive/DI/trainTWEETS.csv', index_col=0)
# trainDATA.rename(columns={"label": "label", "text": "tweet"}, inplace=True)
# trainDATA

In [ ]:
# testDATA = pd.read_csv('/content/gdrive/My Drive/DI/testTWEETS.csv', index_col=0)
# testDATA.rename(columns={"label": "label", "text": "tweet"}, inplace=True)
# testDATA

# trainDATA = trainDATA.sample(n = 8500)
# testDATA = testDATA.sample(n = 600, replace=True)

# X = trainDATA.tweet.values
# y = trainDATA.label.values
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=2020)

### A GPU can be added by going to the menu and selecting:
### Runtime -> Change runtime type -> Hardware accelerator: GPU

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
trainDATA = tweets
trainDATA.rename(columns={"label": "label", "text": "tweet"}, inplace=True)

trainDATA

In [ ]:
from gensim.utils import simple_preprocess

top_data_df_small = trainDATA

top_data_df_small['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in top_data_df_small['tweet']] 
print(top_data_df_small['tokenized_text'].head(10))

In [ ]:
def split_train_test(top_data_df_small, test_size=0.3, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(
          top_data_df_small[['label', 'tweet', 'tokenized_text']], 
          top_data_df_small['label'], 
          shuffle=shuffle_state,
          test_size=test_size, 
          random_state=15
    )
    print("Value counts for Train sentiments")
    print(Y_train.value_counts())
    print("Value counts for Test sentiments")
    print(Y_test.value_counts())

    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(top_data_df_small)

X_train = X_train.sample(n = 5000)

In [ ]:
# Use cuda if present
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)

In [ ]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        
        # Linear function 1: vocab_size --> 500
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity 1
        self.relu1 = nn.ReLU()

        # Linear function 2: 500 --> 500
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        
        # Non-linearity 2
        self.relu2 = nn.ReLU()

        # Linear function 3 (readout): 500 --> 3
        self.fc3 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)

        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)

        # Linear function 3 (readout)
        out = self.fc3(out)

        return F.softmax(out, dim=1)

In [ ]:
from gensim import corpora

# Function to return the dictionary either with padding word or without padding
def make_dict(top_data_df_small, padding=True):
    if padding:
        print("Dictionary with padded token added")
        review_dict = corpora.Dictionary([['pad']])
        review_dict.add_documents(top_data_df_small['tokenized_text'])
    else:
        print("Dictionary without padding")
        review_dict = corpora.Dictionary(top_data_df_small['tokenized_text'])
    return review_dict

# Make the dictionary without padding for the basic models
# review_dict = make_dict(top_data_df_small, padding=False)
review_dict = make_dict(top_data_df_small)
review_dict

In [ ]:
VOCAB_SIZE = len(review_dict)
NUM_LABELS = 3

# Function to make bow vector to be used as input to network
def make_bow_vector(review_dict, sentence):
    vec = torch.zeros(VOCAB_SIZE, dtype=torch.float64, device=device)
    for word in sentence:
        vec[review_dict.token2id[word]] += 1
    return vec.view(1, -1).float()

In [ ]:
def make_target(label):
    if label == 0:
        return torch.tensor([0], dtype=torch.long, device=device)
    elif label == 1:
        return torch.tensor([1], dtype=torch.long, device=device)

In [ ]:
VOCAB_SIZE  = len(review_dict)
input_dim   = VOCAB_SIZE
hidden_dim  = 500
output_dim  = 2
num_epochs  = 15

ff_nn_bow_model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)
ff_nn_bow_model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer     = optim.SGD(ff_nn_bow_model.parameters(), lr=0.001)

losses = []
iter = []

for epoch in range(num_epochs):
    # if (epoch+1) % 5 == 0:
    print("Epoch completed: " + str(epoch+1))
    train_loss = 0
    for index, row in X_train.iterrows():
        
        # Clearing the accumulated gradients
        optimizer.zero_grad()

        # Make the bag of words vector for stemmed tokens 
        bow_vec = make_bow_vector(review_dict, row['tokenized_text'])
       
        # Forward pass to get output
        probs = ff_nn_bow_model(bow_vec)

        # Get the target label
        target = make_target(Y_train['label'][index])

        # Calculate Loss: softmax --> cross entropy loss
        loss = loss_function(probs, target)

        # Accumulating the loss over time
        train_loss += loss.item()

        losses.append( train_loss / len(X_train) )
        iter.append( epoch+1 )

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

    train_loss = 0

plt.plot(losses, label='Training loss')
plt.plot(iter, label='Iters')
plt.legend()
plt.show()

In [ ]:
bow_ff_nn_predictions = []
original_lables_ff_bow = []
with torch.no_grad():
    for index, row in X_test.iterrows():
        bow_vec = make_bow_vector(review_dict, row['tokenized_text'])
        probs = ff_nn_bow_model(bow_vec)
        bow_ff_nn_predictions.append(torch.argmax(probs, dim=1).cpu().numpy()[0])
        original_lables_ff_bow.append(make_target(Y_test['label'][index]).cpu().numpy()[0])

print(classification_report(original_lables_ff_bow, bow_ff_nn_predictions))
print("Accuracy Score -> ",   accuracy_score(   original_lables_ff_bow, bow_ff_nn_predictions))
print("Precision Score -> ",  precision_score(  original_lables_ff_bow, bow_ff_nn_predictions, average='macro')*100)
print("Recall Score -> ",     recall_score(     original_lables_ff_bow, bow_ff_nn_predictions, average='macro')*100)
print("F-Measure Score -> ",  f1_score(         original_lables_ff_bow, bow_ff_nn_predictions, average='macro')*100)

In [ ]:
end_timeTOTAL = time.time()

mins, secs = epoch_time(start_timeTOTAL, end_timeTOTAL)

print(f'Total Time: {mins}m {secs}s')

### A GPU can be added by going to the menu and selecting:
### Runtime -> Change runtime type -> Hardware accelerator: GPU

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
trainDATA = tweets
trainDATA.rename(columns={"label": "label", "text": "tweet"}, inplace=True)

trainDATA

In [ ]:
from gensim.utils import simple_preprocess

top_data_df_small = trainDATA

top_data_df_small['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in top_data_df_small['tweet']] 
print(top_data_df_small['tokenized_text'].head(10))

In [ ]:
def split_train_test(top_data_df_small, test_size=0.3, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(
        top_data_df_small[['label', 'tweet', 'tokenized_text']], 
        top_data_df_small['label'], 
        shuffle=shuffle_state,
        test_size=test_size, 
        random_state=15)
    print("Value counts for Train sentiments")
    print(Y_train.value_counts())
    print("Value counts for Test sentiments")
    print(Y_test.value_counts())

    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(top_data_df_small)

X_train = X_train.sample(n = 10000)

In [ ]:
# Use cuda if present
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)

In [ ]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        
        # Linear function 1: vocab_size --> 500
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity 1
        # self.relu1 = nn.ReLU()
        self.relu1 = nn.Sigmoid()

        # Linear function 2: 500 --> 500
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        # self.relu2 = nn.ReLU()
        self.relu2 = nn.Sigmoid()

        # Linear function 3 (readout): 500 --> 3
        self.fc3 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)

        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)

        # Linear function 3 (readout)
        out = self.fc3(out)

        return F.softmax(out, dim=1)

In [ ]:
from gensim import corpora

# Function to return the dictionary either with padding word or without padding
def make_dict(top_data_df_small, padding=True):
    if padding:
        print("Dictionary with padded token added")
        review_dict = corpora.Dictionary([['pad']])
        review_dict.add_documents(top_data_df_small['tokenized_text'])
    else:
        print("Dictionary without padding")
        review_dict = corpora.Dictionary(top_data_df_small['tokenized_text'])
    return review_dict

# Make the dictionary without padding for the basic models
review_dict = make_dict(top_data_df_small, padding=False)
review_dict

In [ ]:
VOCAB_SIZE = len(review_dict)
NUM_LABELS = 3

# Function to make bow vector to be used as input to network
def make_bow_vector(review_dict, sentence):
    vec = torch.zeros(VOCAB_SIZE, dtype=torch.float64, device=device)
    for word in sentence:
        vec[review_dict.token2id[word]] += 1
    return vec.view(1, -1).float()

In [ ]:
def make_target(label):
    if label == 0:
        return torch.tensor([0], dtype=torch.long, device=device)
    elif label == 1:
        return torch.tensor([1], dtype=torch.long, device=device)

In [ ]:
VOCAB_SIZE = len(review_dict)

input_dim = VOCAB_SIZE
hidden_dim = 500
output_dim = 2
num_epochs = 15

ff_nn_bow_model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)
ff_nn_bow_model.to(device)

loss_function = nn.CrossEntropyLoss()
# loss_function = nn.MSELoss()

# optimizer = optim.SGD(ff_nn_bow_model.parameters(), lr=0.001)
optimizer = AdamW(
        ff_nn_bow_model.parameters(),
        lr=5e-5,    # Default learning rate
        eps=1e-8    # Default epsilon value
    )

In [ ]:
losses = []
iter = []
# Start training
for epoch in range(num_epochs):
    # if (epoch+1) % 5 == 0:
    print("Epoch completed: " + str(epoch+1))
    train_loss = 0
    for index, row in X_train.iterrows():
        # Clearing the accumulated gradients
        optimizer.zero_grad()

        # Make the bag of words vector for stemmed tokens 
        bow_vec = make_bow_vector(review_dict, row['tokenized_text'])
       
        # Forward pass to get output
        probs = ff_nn_bow_model(bow_vec)

        # Get the target label
        target = make_target(Y_train['label'][index])

        # Calculate Loss: softmax --> cross entropy loss
        loss = loss_function(probs, target)
        # Accumulating the loss over time
        train_loss += loss.item()

        losses.append( train_loss / len(X_train) )
        iter.append( epoch+1 )

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

    train_loss = 0

plt.plot(losses, label='Training loss')
plt.plot(iter, label='Iters')
plt.legend()
plt.show()

In [ ]:
bow_ff_nn_predictions = []
original_lables_ff_bow = []
with torch.no_grad():
    for index, row in X_test.iterrows():
        bow_vec = make_bow_vector(review_dict, row['tokenized_text'])
        probs = ff_nn_bow_model(bow_vec)
        bow_ff_nn_predictions.append(torch.argmax(probs, dim=1).cpu().numpy()[0])
        original_lables_ff_bow.append(make_target(Y_test['label'][index]).cpu().numpy()[0])

print(classification_report(original_lables_ff_bow, bow_ff_nn_predictions))
print("Accuracy Score -> ",   accuracy_score(   original_lables_ff_bow, bow_ff_nn_predictions))
print("Precision Score -> ",  precision_score(  original_lables_ff_bow, bow_ff_nn_predictions, average='macro')*100)
print("Recall Score -> ",     recall_score(     original_lables_ff_bow, bow_ff_nn_predictions, average='macro')*100)
print("F-Measure Score -> ",  f1_score(         original_lables_ff_bow, bow_ff_nn_predictions, average='macro')*100)

In [ ]:
end_timeTOTAL = time.time()

mins, secs = epoch_time(start_timeTOTAL, end_timeTOTAL)

print(f'Total Time: {mins}m {secs}s')